In [3]:
from pandas import read_csv
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator
# from pandas.plotting import scatter_matrix
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler       # scaling data
from sklearn.model_selection import train_test_split # splitting data
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV     # for grid search
from sklearn.pipeline import make_pipeline           # for making pipelines
# from sklearn.neighbors import KNeighborsRegressor    # regressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings('ignore')


In [16]:
#read in the en csv files 
all_data = read_csv('data/data.csv')
all_data.isnull().sum()
all_data.head()

,Date,"5 Home Sold As Foreclosure - Ratio - All Homes Name: INDICATOR, dtype: object","6 Inventory Measure (Public) Name: INDICATOR, dtype: object","7 Inventory Measure - SSA (Public) Name: INDICATOR, dtype: object","9 Listings Price Cuts (SA) All Homes Name: INDICATOR, dtype: object","11 Listings Price Cuts (SA) Condominiums Name: INDICATOR, dtype: object","13 Listings Price Cuts (SA) Single-Family Residence Name: INDICATOR, dtype: object","16 Median Listing Price - All Homes Name: INDICATOR, dtype: object","17 Median Listing Price - Condo/Co-op Name: INDICATOR, dtype: object","18 Median Listing Price - Duplex/Triplex Name: INDICATOR, dtype: object",...,"92 Zillow Home Value Index - Single-Family Residence Name: INDICATOR, dtype: object","93 Zillow Home Value Index - Three Bedrooms Name: INDICATOR, dtype: object","94 Zillow Home Value Index - Top Tier - Year-Over... Name: INDICATOR, dtype: object","95 Zillow Home Value Index - Top Tier Name: INDICATOR, dtype: object","96 Zillow Home Value Index - Two Bedrooms Name: INDICATOR, dtype: object","98 Zillow Rental Index - All Homes Plus Multi-Family Name: INDICATOR, dtype: object","100 Zillow Rental Index - All Homes Name: INDICATOR, dtype: object","102 Zillow Rental Index - Multi-Family Residence Name: INDICATOR, dtype: object","104 Zillow Rental Index - Single-Family Residence Name: INDICATOR, dtype: object","105 Zillow Rental Index Per Square Foot - All Homes Name: INDICATOR, dtype: object"
0,1/31/1998,1.0438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,196200,198000,0.142528,297400,164200,NaN,NaN,NaN,NaN,NaN
1,2/28/1998,0.8824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,198800,200700,0.143781,300700,166000,NaN,NaN,NaN,NaN,NaN
2,3/31/1998,0.6395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,201400,203400,0.145386,304100,167700,NaN,NaN,NaN,NaN,NaN
3,4/30/1998,0.5347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,204000,206100,0.146532,307500,169400,NaN,NaN,NaN,NaN,NaN
4,5/31/1998,0.6916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,206800,208900,0.147547,311100,171100,NaN,NaN,NaN,NaN,NaN
